### Search Engine With Tools And Agents

In [1]:
#initialise env variables
import os
from dotenv import load_dotenv
load_dotenv()
import certifi  # Import certifi

#map your envirnment api key with the api key present in dotenv file 
openai_api_key=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

# Set SSL certificate file path using certifi
os.environ["SSL_CERT_FILE"] = certifi.where()

In [2]:
## arxiv ---# API for accessing research papers from Arxiv
#tool creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

* using inbuilt tool of wikipedia

In [3]:
#initialise wiki api wrapper
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

#initialise wiki runner
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

* using inbuilt tool of arxiv

In [6]:
#initilise the arxic wrapper
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)

#initialise arxiv runner
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


* Tools to use 

In [7]:
tools=[arxiv,wiki]

* Custom tools[RAG Tool]

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
#data ingestion
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
#data transformation
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
#instantiate embedding model 
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")
#vector store
vectorstore=FAISS.from_documents(documents=splits, embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028A66F67EB0>, search_kwargs={})

* convert this retriever into a tool

In [10]:
from langchain.tools.retriever import create_retriever_tool

#initialise tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

Final Tool kit

In [11]:
tools=[arxiv,wiki,retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\code\\Projects\\Generative AI Projects\\Langchain Projects\\Q&A ChatBot\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000028A4E449B40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores

#### Now, Run all these tools with Agents and LLM models

* Create Agent - go to https://python.langchain.com/api_reference/langchain/agents/langchain.agents.openai_tools.base.create_openai_tools_agent.html

In [12]:
##Tools--> AgentExecuter
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
# Choose the LLM Groq Ollama Model that will drive the agent
from langchain_groq import ChatGroq
llm=ChatGroq(api_key=groq_api_key,model_name="Llama3-8b-8192")

In [14]:
# Construct the OpenAI tools agent
from langchain.agents import create_openai_tools_agent

#initialise the agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

* Make Agent Executer to run this Agent

In [15]:
from langchain.agents import AgentExecutor

#initialise agent executor
agent_excecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_excecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_excecutor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main conten

{'input': 'Tell me about Langsmith',
 'output': 'I can use the information yielded by the tool call to respond to your question about Langsmith.\n\nLangsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. Langsmith provides various features, including observability, evaluation, and prompt engineering.\n\nObservability is a key aspect of Langsmith, allowing developers to analyze traces, configure metrics, and set up dashboards and alerts. This helps developers identify issues and optimize their applications.\n\nEvaluations are also crucial in Langsmith, enabling developers to test and optimize their applications. Langsmith provides tools for building and running high-quality evaluations, making it easy to assess the performance of applications and collect human feedback to improve them.\n\nPrompt engineering is another importan

In [17]:
agent_excecutor.invoke({"input":"What is Machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, asIt seems like the tool call yiel

{'input': 'What is Machine learning',
 'output': "It seems like the tool call yielded the same result again. Since we don't have any new information, I'll provide a direct response.\n\nMachine learning is a field of study in artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make decisions without being explicitly programmed. The goal of machine learning is to enable machines to learn from experience and improve their performance over time.\n\nThe field of machine learning has many applications, including image and speech recognition, natural language processing, and predictive analytics. It is a rapidly growing field with many potential applications in areas such as healthcare, finance, and transportation.\n\nI hope this helps! Let me know if you have any further questions."}

In [18]:
agent_excecutor.invoke({"input":"What is this research paper 1706.03762 all about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper "Attention Is All You Need" (1706.03762) is about the development of a new sequence transduction model that replaces the traditional recurrent neural network (RNN) and convolutional neural network (CNN) based models. The authors propose a new architecture called the Transformer, which uses self-attention mechanisms to process input sequences. The Transformer is capable of handling long-range dependencies and parallelizing the processing of sequences, making it more efficient and effective than traditional models.

> Finished chain.


{'input': 'What is this research paper 1706.03762 all about?',
 'output': 'The research paper "Attention Is All You Need" (1706.03762) is about the development of a new sequence transduction model that replaces the traditional recurrent neural network (RNN) and convolutional neural network (CNN) based models. The authors propose a new architecture called the Transformer, which uses self-attention mechanisms to process input sequences. The Transformer is capable of handling long-range dependencies and parallelizing the processing of sequences, making it more efficient and effective than traditional models.'}